<a href="https://colab.research.google.com/github/monjurkuet/yt-crawler/blob/main/crawl_all_yt_videos_from_channel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import userdata # Keep for direct Colab execution, but prefer os.environ

# Define the CHANNEL_ID for the YouTube channel
CHANNEL_ID = 'UCHOP_YfwdMk5hpxbugzC1wA'

class ConfigManager:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(ConfigManager, cls).__new__(cls)
            cls._instance._load_config()
        return cls._instance

    def _load_config(self):
        # Helper to get config, prioritizing environment variables
        def get_config_value(key):
            return os.environ.get(key) or userdata.get(key)

        # YouTube API Credentials
        self.API_KEY = get_config_value('API_KEY')

        # SSH Tunnel & MySQL Database Credentials
        self.SSH_HOST = get_config_value('SSH_HOST')
        self.SSH_USERNAME = 'administrator' # This is often a fixed username for cloud VMs
        self.SSH_PRIVATEKEY_PATH = get_config_value('SSH_PRIVATEKEY_PATH') # Path to private key on Colab
        self.LOCAL_PORT = 3307 # Local port for SSH tunnel
        self.REMOTE_MYSQL_HOST = '127.0.0.1'
        self.REMOTE_MYSQL_PORT = 3306
        self.DATABASE_NAME = get_config_value('DATABASE_NAME')
        self.DATABASE_PASSWORD = get_config_value('DATABASE_PASSWORD')

    def get_config(self):
        return {
            'API_KEY': self.API_KEY,
            'SSH_HOST': self.SSH_HOST,
            'SSH_USERNAME': self.SSH_USERNAME,
            'SSH_PRIVATEKEY_PATH': self.SSH_PRIVATEKEY_PATH,
            'LOCAL_PORT': self.LOCAL_PORT,
            'REMOTE_MYSQL_HOST': self.REMOTE_MYSQL_HOST,
            'REMOTE_MYSQL_PORT': self.REMOTE_MYSQL_PORT,
            'DATABASE_NAME': self.DATABASE_NAME,
            'DATABASE_PASSWORD': self.DATABASE_PASSWORD
        }

# Example usage (for testing purposes, not part of the module's core function)
if __name__ == '__main__':
    config = ConfigManager()
    #print("API Key:", config.API_KEY)
    #print("SSH Host:", config.SSH_HOST)
    #print("Database Name:", config.DATABASE_NAME)


In [12]:
from googleapiclient.discovery import build

print("--- Initializing YouTube API Client ---")

# Build the YouTube API service object
youtube = build('youtube', 'v3', developerKey=config.API_KEY)

print("YouTube API client initialized successfully.")

--- Initializing YouTube API Client ---
YouTube API client initialized successfully.


In [13]:
print("--- Fetching Uploads Playlist ID ---")

# 2. Call the channels().list() method with 'contentDetails' part
request = youtube.channels().list(
    part='contentDetails',
    id=CHANNEL_ID
)

# 3. Execute the API request
response = request.execute()

# 4. From the API response, extract the 'uploads' playlist ID
#    This is typically found at response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
uploads_playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

# 5. Print the retrieved ID
print(f"'Uploads' Playlist ID for channel {CHANNEL_ID}: {uploads_playlist_id}")
print("Uploads playlist ID fetched successfully.")

--- Fetching Uploads Playlist ID ---
'Uploads' Playlist ID for channel UCHOP_YfwdMk5hpxbugzC1wA: UUHOP_YfwdMk5hpxbugzC1wA
Uploads playlist ID fetched successfully.


In [ ]:
from tqdm.notebook import tqdm

# 1. Initialize an empty list to store all collected video details
all_video_items = []

# 2. Initialize next_page_token to None
next_page_token = None

# 3. Start a while loop for pagination with tqdm for progress bar
with tqdm(total=None, desc="Collecting videos") as pbar:
    while True:
        # 4. Construct the request
        request = youtube.playlistItems().list(
            playlistId=uploads_playlist_id,
            part='snippet',
            maxResults=50, # Maximum allowed per request
            pageToken=next_page_token
        )

        # 5. Execute the API request
        response = request.execute()

        # 6. Extend the all_video_items list with the 'items' from the API response
        items = response.get('items', [])
        all_video_items.extend(items)

        # Update the progress bar
        pbar.update(len(items))

        # 7. Update next_page_token
        next_page_token = response.get('nextPageToken')

        # 8. If next_page_token is None, break the loop
        if not next_page_token:
            break

# The total number of videos collected is implicitly known by the length of all_video_items
# as the progress bar already shows the progress.


In [18]:
# preview data
all_video_items[0]

{'kind': 'youtube#playlistItem',
 'etag': 'tkdVloFG1CJM5HpHMcnHPUVM0KA',
 'id': 'VVVIT1BfWWZ3ZE1rNWhweGJ1Z3pDMXdBLlhIcmZ1clNtOVhj',
 'snippet': {'publishedAt': '2025-12-06T11:01:32Z',
  'channelId': 'UCHOP_YfwdMk5hpxbugzC1wA',
  'title': 'BTC Structure Breakdown',
  'description': 'In this video, I explain why 107k was an important level in my own Bitcoin analysis and how I interpreted the shift in structure once that support was lost. I share how the level had acted as support multiple times and why a break below it signalled a potential change in market behavior.\nI also show how support can flip into resistance and why the retest offered a clean technical reaction. \n\n🚀 Learn to read the markets like a pro: Join 40,000+ traders now → https://chartchampions.com\n\n📈 Trade Smarter. Learn Faster. Become a Chart Champion.\nChart Champions is the world’s most active online trading education community, helping traders from beginners to experienced market analysts improve their skills.\n\

In [19]:
print("--- Extracting and Displaying Video Information ---")

# 1. Initialize an empty list to store extracted video information
extracted_video_info = []

# 2. Iterate through each item in the all_video_items list
for item in all_video_items:
    snippet = item.get('snippet', {})

    # 3. Extract key information, providing default empty strings for missing fields
    channel_id = snippet.get('channelId', '')
    title = snippet.get('title', '')
    description = snippet.get('description', '')
    published_at = snippet.get('publishedAt', '')
    video_id = snippet.get('resourceId', {}).get('videoId', '')

    # 4. Append a dictionary containing the extracted information to the list
    extracted_video_info.append({
        'channelId': channel_id,
        'title': title,
        'description': description,
        'videoId': video_id,
        'publishedAt': published_at
    })

# 5. Print a summary showing the total number of videos extracted
print(f"Total videos with extracted information: {len(extracted_video_info)}")

# 6. If extracted_video_info is not empty, print a sample of the data
if extracted_video_info:
    print("\n--- Sample of Extracted Video Details (First 5) ---")
    for i, video in enumerate(extracted_video_info[:5]):
        print(f"\nSample {i+1}:")
        print(f"  Channel ID: {video['channelId']}")
        print(f"  Title: {video['title']}")
        print(f"  Video ID: {video['videoId']}")
        print(f"  Published At: {video['publishedAt']}")
        print(f"  Description (first 100 chars): {video['description'][:100]}...")
else:
    print("\nNo video information was extracted.")

print("--- Video information extraction complete ---")

--- Extracting and Displaying Video Information ---
Total videos with extracted information: 1701

--- Sample of Extracted Video Details (First 5) ---

Sample 1:
  Channel ID: UCHOP_YfwdMk5hpxbugzC1wA
  Title: BTC Structure Breakdown
  Video ID: XHrfurSm9Xc
  Published At: 2025-12-06T11:01:32Z
  Description (first 100 chars): In this video, I explain why 107k was an important level in my own Bitcoin analysis and how I interp...

Sample 2:
  Channel ID: UCHOP_YfwdMk5hpxbugzC1wA
  Title: Why Missing a Trade Doesn’t Matter #TradingMindset
  Video ID: nWqtaDd8ZFI
  Published At: 2025-12-03T16:47:24Z
  Description (first 100 chars): Missing a long trade doesn’t affect me. Emotional detachment is one of the most important skills in ...

Sample 3:
  Channel ID: UCHOP_YfwdMk5hpxbugzC1wA
  Title: My BTC trading plan for the Bear market
  Video ID: G4BEyczW-J4
  Published At: 2025-11-30T16:00:25Z
  Description (first 100 chars): I share my current approach to navigating the Bitcoin bear market a

In [20]:
!pip install mysql-connector-python sshtunnel paramiko==3.4.0 tenacity

In [ ]:
import mysql.connector
from sshtunnel import SSHTunnelForwarder, BaseSSHTunnelForwarderError
import warnings
import paramiko
import logging
from tenacity import (
    retry,
    stop_after_attempt,
    wait_exponential,
    retry_if_exception_type,
    after_log
)
from tqdm.notebook import tqdm # Import tqdm for progress bars

# Suppress Paramiko UserWarning about missing cryptography library
warnings.filterwarnings('ignore', category=UserWarning, module='paramiko')

class DBConnector:
    def __init__(self):
        self.config = ConfigManager()
        self.ssh_tunnel = None
        self.mysql_conn = None
        self.logger = self._setup_logging()

        self.create_table_sql = """
CREATE TABLE IF NOT EXISTS youtube_videos (
    video_id VARCHAR(255) PRIMARY KEY,
    channel_id VARCHAR(255),
    published_at DATETIME,
    title VARCHAR(255),
    description TEXT,
    tags TEXT, -- Storing tags as a JSON string
    category_id VARCHAR(50),
    default_language VARCHAR(10),
    duration VARCHAR(50), -- Storing YouTube's ISO 8601 duration format
    caption BOOLEAN,
    view_count BIGINT,
    like_count BIGINT,
    comment_count BIGINT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
);
"""

    def _setup_logging(self):
        logger = logging.getLogger('db_connector')
        if not logger.handlers: # Prevent adding multiple handlers
            handler = logging.StreamHandler() # Or a file handler
            formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            handler.setFormatter(formatter)
            logger.addHandler(handler)
            logger.setLevel(logging.INFO)
        return logger

    # Helper for retry condition
    def _is_connection_error(self, exception):
        return isinstance(exception, (
            mysql.connector.Error,
            paramiko.SSHException,
            BaseSSHTunnelForwarderError
        ))

    @retry(
        wait=wait_exponential(multiplier=1, min=2, max=10),
        stop=stop_after_attempt(3),
        retry=retry_if_exception_type((mysql.connector.Error, paramiko.SSHException, BaseSSHTunnelForwarderError)),
        after=after_log(logging.getLogger('db_connector'), logging.WARNING)
    )
    def establish_connection(self):
        self.close_connection() # Ensure previous connections are closed before retrying
        try:
            # Establish SSH tunnel
            self.logger.info("Attempting to establish SSH tunnel...")
            self.ssh_tunnel = SSHTunnelForwarder(
                (self.config.SSH_HOST, 22),
                ssh_username=self.config.SSH_USERNAME,
                ssh_pkey=self.config.SSH_PRIVATEKEY_PATH,
                remote_bind_address=(self.config.REMOTE_MYSQL_HOST, self.config.REMOTE_MYSQL_PORT),
                local_bind_address=('0.0.0.0', self.config.LOCAL_PORT)
            )
            self.ssh_tunnel.start()
            self.logger.info(f"SSH tunnel established on local port {self.ssh_tunnel.local_bind_port}")

            # Connect to MySQL
            self.logger.info("Attempting to connect to MySQL database...")
            self.mysql_conn = mysql.connector.connect(
                host='localhost',
                port=self.ssh_tunnel.local_bind_port,
                user='adminuser',
                password=self.config.DATABASE_PASSWORD,
                database=self.config.DATABASE_NAME
            )
            if self.mysql_conn.is_connected():
                self.logger.info("Successfully connected to MySQL database.")
                return True
            else:
                self.logger.error("Failed to connect to MySQL database after SSH tunnel.")
                self.close_connection() # Close partially established connections
                return False

        except (mysql.connector.Error, paramiko.SSHException, BaseSSHTunnelForwarderError) as e:
            self.logger.error(f"Connection error during establish_connection: {e}")
            self.close_connection()
            raise # Re-raise to trigger retry
        except Exception as e:
            self.logger.critical(f"An unexpected critical error occurred during connection establishment: {e}")
            self.close_connection()
            return False

    def create_table(self):
        if self.mysql_conn and self.mysql_conn.is_connected():
            try:
                cursor = self.mysql_conn.cursor()
                self.logger.info("Executing CREATE TABLE statement...")
                cursor.execute(self.create_table_sql)
                self.mysql_conn.commit()
                cursor.close()
                self.logger.info("Table 'youtube_videos' ensured.")
                return True
            except mysql.connector.Error as err:
                if err.errno == 1050: # Table already exists
                    self.logger.info("Table 'youtube_videos' already exists. Skipping creation.")
                else:
                    self.logger.error(f"Error creating table: {err}")
                return False
            except Exception as e:
                self.logger.error(f"An unexpected error occurred during table creation: {e}")
                return False
        else:
            self.logger.warning("Cannot create table: MySQL connection is not active.")
        return False

    def insert_video_data(self, video_data):
        if not self.mysql_conn or not self.mysql_conn.is_connected():
            self.logger.error("No active MySQL connection to insert data.")
            return False

        try:
            cursor = self.mysql_conn.cursor()
            # SQL for INSERT IGNORE. Only inserting the fields we have.
            # The other fields will be NULL or default as per table definition.
            insert_sql = """
            INSERT IGNORE INTO youtube_videos (video_id, channel_id, published_at, title, description)
            VALUES (%s, %s, %s, %s, %s)
            """

            # Prepare data for insertion
            records_to_insert = []
            for video in video_data:
                records_to_insert.append((
                    video.get('videoId'),
                    video.get('channelId'),
                    video.get('publishedAt'),
                    video.get('title'),
                    video.get('description')
                ))

            if not records_to_insert:
                self.logger.info("No records to insert.")
                return True

            BATCH_SIZE = 1000  # Define an appropriate batch size
            total_records = len(records_to_insert)
            inserted_rows_count = 0

            self.logger.info(f"Starting batch insertion for {total_records} video records.")
            for i in tqdm(range(0, total_records, BATCH_SIZE), desc="Inserting video data batches"):
                batch = records_to_insert[i:i + BATCH_SIZE]
                cursor.executemany(insert_sql, batch)
                self.mysql_conn.commit()
                inserted_rows_count += cursor.rowcount

            self.logger.info(f"Video data insertion complete. Total new records processed: {inserted_rows_count} (inserted or ignored duplicates).")
            cursor.close()
            return True
        except mysql.connector.Error as err:
            self.logger.error(f"Error inserting video data: {err}")
            self.mysql_conn.rollback() # Rollback if an error occurs
            return False
        except Exception as e:
            self.logger.error(f"An unexpected error occurred during data insertion: {e}")
            return False

    def close_connection(self):
        if self.mysql_conn and self.mysql_conn.is_connected():
            self.mysql_conn.close()
            self.mysql_conn = None
            self.logger.info("MySQL connection closed.")
        if self.ssh_tunnel and self.ssh_tunnel.is_active:
            self.ssh_tunnel.stop()
            self.ssh_tunnel = None
            self.logger.info("SSH tunnel stopped.")

# Example usage
if __name__ == '__main__':
    db_connector = DBConnector()
    if db_connector.establish_connection():
        db_connector.create_table()

        # Assuming 'extracted_video_info' is available from previous steps

        if extracted_video_info:
            db_connector.insert_video_data(extracted_video_info)
        else:
            db_connector.logger.info("No video data available to insert.")

        db_connector.close_connection()
    else:
        db_connector.logger.error("Failed to connect to database.")
